## Working with MCNP input files

### Quick general overview

In [1]:
# Import the input module
from f4enix.input.MCNPinput import Input

# Load the input file
inp_file = 'ITER_1D'
inp = Input.from_input(inp_file)

In [2]:
# Here are the main attributes of the input
print(inp.cells)
print(inp.surfs)
print(inp.materials)  # datacard class that have their own complete parser
print(inp.transformations)  # uncomplete parser (no transformations in this inp)
print(inp.other_data)  # treated as general data cards, no parser

{'1': <numjuggler.parser.Card object at 0x000001E02F0BDED0>, '2': <numjuggler.parser.Card object at 0x000001E02F0BDF00>, '3': <numjuggler.parser.Card object at 0x000001E02F0BDEA0>, '4': <numjuggler.parser.Card object at 0x000001E02F0BDE40>, '5': <numjuggler.parser.Card object at 0x000001E02F0BDE70>, '6': <numjuggler.parser.Card object at 0x000001E02F0BDF60>, '7': <numjuggler.parser.Card object at 0x000001E02F0BDF30>, '8': <numjuggler.parser.Card object at 0x000001E02F0BDF90>, '9': <numjuggler.parser.Card object at 0x000001E02F0BDFC0>, '10': <numjuggler.parser.Card object at 0x000001E02F0BDFF0>, '11': <numjuggler.parser.Card object at 0x000001E02F0BE020>, '12': <numjuggler.parser.Card object at 0x000001E02F0BE050>, '13': <numjuggler.parser.Card object at 0x000001E02F0BE080>, '14': <numjuggler.parser.Card object at 0x000001E02F0BE0B0>, '15': <numjuggler.parser.Card object at 0x000001E02F0BE0E0>, '16': <numjuggler.parser.Card object at 0x000001E02F0BE110>, '17': <numjuggler.parser.Card ob

In [3]:
# Get a specific card
print(inp.get_cells_by_id([2]))  # cell n.2
print(inp.get_surfs_by_id([1, 2]))  # surfaces n.1 and 2
print(inp.get_cells_by_matID('13'))  # cells to which material 13 is assigned
print(inp.get_materials_subset(['m2', 'm3']))  # materials M2 and M3
print(inp.get_data_cards('SDEF'))  # generic data cards

{'2': <numjuggler.parser.Card object at 0x000001E02F0BDF00>}
{'1': <numjuggler.parser.Card object at 0x000001E02F0BF2E0>, '2': <numjuggler.parser.Card object at 0x000001E02F0BF6D0>}
{'2': <numjuggler.parser.Card object at 0x000001E02F110E80>, '3': <numjuggler.parser.Card object at 0x000001E02F110CA0>, '4': <numjuggler.parser.Card object at 0x000001E02F110C40>, '5': <numjuggler.parser.Card object at 0x000001E02F110820>, '6': <numjuggler.parser.Card object at 0x000001E02F110DF0>, '7': <numjuggler.parser.Card object at 0x000001E02F110D90>, '8': <numjuggler.parser.Card object at 0x000001E02F1107F0>, '9': <numjuggler.parser.Card object at 0x000001E02F112590>, '10': <numjuggler.parser.Card object at 0x000001E01EDB34C0>, '11': <numjuggler.parser.Card object at 0x000001E01EDB3520>, '12': <numjuggler.parser.Card object at 0x000001E01EDB3A60>, '13': <numjuggler.parser.Card object at 0x000001E01EDB3AF0>, '14': <numjuggler.parser.Card object at 0x000001E01EDB3880>, '15': <numjuggler.parser.Card ob

### Working with surfaces and cells

A common example may be the need to modify some cells attributes based on an arbitrary logic. Here is an example where the density of cells is reduced by a factor depending on the material. If a constant factor has to be used, consider the ``Input.scale_densities()`` method

In [4]:
from copy import deepcopy
import tempfile  # To have a scratch directory for the example
import os

# It may be useful to preserve the original input
inp2 = deepcopy(inp)

# Set some density correction factors for specific materials
density_factors = {61: 1.1, 63: 10}

# Cycle on the cells dictionary
for idx_cell, cell in inp2.cells.items():
    # Change density based on material
    # mat = cell._get_value_by_type('mat')
    mat = cell.get_m()
    # get the density of the cell
    rho = cell.get_d()
    if mat in [61, 63]:
        cell.set_d(str(rho*density_factors[mat]))
    elif mat == 0:
        pass  # do not change density of void cells
    else:
        # scale all other densitites by a constant factor
        cell.set_d(str(rho*0.5))

# Print the modified file
outfile = os.path.join(tempfile.gettempdir(), 'new_input.i')
inp2.write(outfile)
print(outfile)

C:\Users\laghida\AppData\Local\Temp\new_input.i


This input parser is built on [numjuggler](https://pypi.org/project/numjuggler/), hence every card is a ``numjuggler.parser.Card``. The recommended way to get and modify fields of these cards are to use the ``_get_value_by_type()`` and ``_set_value_by_type()`` methods, as in the example above. Since numjuggler is lacking extensive documentation, to understand which methods are available, the ``dir()`` method can be useful, while the ``values`` attributes should be used to verify the available values to get/modify.

In [5]:
print(type(cell))
print(dir(cell))
cell.values

<class 'numjuggler.parser.Card'>
['_Card__cr', '_Card__d', '_Card__f', '_Card__i', '_Card__m', '_Card__st', '_Card__u', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_get_value_by_type', '_protect_nums', '_set_value_by_type', 'apply_map', 'card', 'cstrg', 'ctype', 'debug', 'dtype', 'geom_prefix', 'geom_suffix', 'get_d', 'get_f', 'get_geom', 'get_imp', 'get_input', 'get_m', 'get_refcells', 'get_u', 'get_values', 'hidden', 'input', 'lines', 'name', 'pos', 'print_debug', 'remove_fill', 'remove_spaces', 'set_d', 'template', 'values']


[(106, 'cel'),
 (0, 'mat'),
 ('', '#gpr'),
 (55, 'sur'),
 (54, 'sur'),
 (56, 'sur'),
 ('', '#gsu')]

#### Replace a material

It is possible to replace a material with another one (and another density) or set it to void. This can be done across the entire input or only in specific universes.

In [6]:
# It may be useful to preserve the original input
inp3 = deepcopy(inp)

print('original card:')
print(inp3.cells['2'].card())
# In this case replaces in the entire input
inp3.replace_material(10, '-1', 13)
print('modified card:')
print(inp3.cells['2'].card())

original card:
2     13  7.2058e-002  -55 56   1  -60   imp:n,p=8388608  

modified card:
2     10  -1  -55 56   1  -60   imp:n,p=8388608           



### Working with materials

#### Get info on the material section of the input and switch fraction

In [7]:
# For many operations on materials a libmanager is needed. For most application
# the default libmanager can be used, check the libmanager documentation for
# more details
from f4enix.input.libmanager import LibManager
libmanager = LibManager()

# Get a summary of the materials defined in the input
df_mat, df_submat = inp.materials.get_info(libmanager)
df_mat


Atom Fraction  Mass Fraction
Material Submaterial Element                              
M1       1           H         2.162999e-02      -0.019046
         2           C         1.892004e-02      -0.198524
         3           N         2.059998e-03      -0.025207
         4           O         2.705996e-02      -0.378226
         5           Mg        1.190000e-03      -0.025268
...                                     ...            ...
M74      1           Si        8.157524e-06      -0.000020
                     Ta        6.330820e-07      -0.000010
                     Ti        2.392466e-06      -0.000010
                     Zr        1.255620e-06      -0.000010
                     W         6.220930e-02      -0.999565

[161 rows x 2 columns]

In [8]:
# here the fraction used in the MCNP input (i.e. either mass or atom) is used
df_submat

Fraction  Sub-Material Fraction  \
Material Submaterial Element                                    
M1       1           H        0.021630               1.000000   
         2           C        0.018920               1.000000   
         3           N        0.002060               1.000000   
         4           O        0.027060               1.000000   
         5           Al       0.003930               0.299542   
...                                ...                    ...   
M74      1           Zr       0.000001               0.000020   
M8       1           Be       0.002970               0.034219   
                     Cu       0.082000               0.944769   
                     Ni       0.001824               0.021012   
M9       1           Be       0.123619               1.000000   

                              Material Fraction  
Material Submaterial Element                     
M1       1           H                 0.256858  
         2           C                 0.224677  
         3           N                 0.024463  
         4           O                 0.321339  
         5           Al                0.046669  
...                                         ...  
M74      1           Zr                0.000020  
M8       1           Be                0.034219  
                     Cu                0.944769  
                     Ni                0.021012  
M9       1           Be                1.000000  

[161 rows x 3 columns]

In [9]:
# switch from atom to mass fraction for all materials
inp2 = deepcopy(inp)

# MaterialSection is implemented as a Sequence, hence, it can be iterated
for material in inp2.materials:
    material.switch_fraction('mass', libmanager)

# verify the change, this time get also the different zaid contribution
df_mat, df_submat = inp2.materials.get_info(libmanager, zaids=True)
df_submat

Fraction  Sub-Material Fraction  \
Material Submaterial Element                                    
M1       1           H       -0.019046               1.000000   
         2           C       -0.198524               1.000000   
         3           N       -0.025207               1.000000   
         4           O       -0.378226               1.000000   
         5           Al      -0.092636               0.294839   
...                                ...                    ...   
M74      1           Zr      -0.000010               0.000010   
M8       1           Be      -0.005008               0.005008   
                     Cu      -0.974957               0.974957   
                     Ni      -0.020035               0.020035   
M9       1           Be      -1.000000               1.000000   

                              Material Fraction  
Material Submaterial Element                     
M1       1           H                 0.019046  
         2           C                 0.198524  
         3           N                 0.025207  
         4           O                 0.378226  
         5           Al                0.092636  
...                                         ...  
M74      1           Zr                0.000010  
M8       1           Be                0.005008  
                     Cu                0.974957  
                     Ni                0.020035  
M9       1           Be                1.000000  

[161 rows x 3 columns]

In [10]:
# the zaids are print only in the df_mat when requested
df_mat

Atom Fraction  Mass Fraction
Material Submaterial Element Isotope                                    
M1       1           H       H-1 [1001]          0.256828      -0.019042
                             H-2 [1002]          0.000030      -0.000004
         2           C       C-12 [6012]         0.222273      -0.196224
                             C-13 [6013]         0.002404      -0.002300
         3           N       N-14 [7014]         0.024374      -0.025109
...                                                   ...            ...
M74      1           W       W-180 [74180]       0.001196      -0.001174
                             W-182 [74182]       0.264161      -0.262157
                             W-183 [74183]       0.142647      -0.142344
                             W-184 [74184]       0.305429      -0.306448
                             W-186 [74186]       0.283399      -0.287442

[506 rows x 2 columns]

#### Translation of the material card to different libraries

In [11]:
# --- Change library for all materials ---
inp2.translate('31c', libmanager)
print('All translated to 81c:')
print(inp2.materials['M8'].to_text())

# --- specify a library for every zaid ---
special_zaids = ['4009', '28061']  # to be translated to specific library
non_special = []
for material in inp2.materials:
    for submaterial in material.submaterials:
        for zaid in submaterial.zaidList:
            zaid_str = zaid.element+zaid.isotope
            if zaid_str not in special_zaids:
                non_special.append(zaid_str)

inp2.translate({'80c': special_zaids, '31c': non_special}, libmanager)
print('\nSpecified library for each isotope:')
print(inp2.materials['M8'].to_text())

# --- specify different libraries depending on the original one ---
# this can be useful when D1S libraries only have to be changed for instance
libs = {'31c': '30c', '80c': '31c'}
inp2.translate(libs, libmanager)
print('\nDifferent libraries based on original one:')
print(inp2.materials['M8'].to_text())

All translated to 81c:
c -------------------------------------------------------------------
M8
c   Cu-Be-Ni:  t.a.d. = 8.6794e-002
      28058.31c       -1.337648E-2     $ Ni-58  AB(%) 66.767
      28060.31c       -5.346947E-3     $ Ni-60  AB(%) 26.688
      28061.31c       -2.697346E-4     $ Ni-61  AB(%) 1.3463
      28062.31c       -7.802716E-4     $ Ni-62  AB(%) 3.8946
      28064.31c       -2.612322E-4     $ Ni-64  AB(%) 1.3039
      29063.31c       -6.676429E-1     $ Cu-63  AB(%) 68.479
      29065.31c       -3.073144E-1     $ Cu-65  AB(%) 31.521
       4009.31c       -5.008062E-3     $ Be-9   AB(%) 100.0
	 plib=84p

Specified library for each isotope:
c -------------------------------------------------------------------
M8
c   Cu-Be-Ni:  t.a.d. = 8.6794e-002
      28058.31c       -1.337648E-2     $ Ni-58  AB(%) 66.767
      28060.31c       -5.346947E-3     $ Ni-60  AB(%) 26.688
      28061.80c       -2.697346E-4     $ Ni-61  AB(%) 1.3463
      28062.31c       -7.802716E-4     $ 

#### Generate new materials from existing ones

In [12]:
# Generate a new material as a composition of existing ones
# The percentages are intended in mass if the fraction is to be expressed in
# mass or in atom otherwise. The new material is created as 'M1'
new_mat = inp2.materials.generate_material(['m2', 'm74'],
                                           [0.2, 0.8],
                                           '31c', libmanager,
                                           fractiontype='mass')
print(new_mat.to_text())

C Material: M2 Percentage: 20.0% (mass)
C Material: M74 Percentage: 80.0% (mass)
M1
c  -----------------------------------------------------------------
C           o-------------------------------------------------------o
c           |    Copper                                             |
c           |    t.a.d. =   8.29204E-02                        M2   |
C           o-------------------------------------------------------o
c
C M2, submaterial 1
      29063.31c       -1.369585E-1     $ Cu-63  AB(%) 68.479
      29065.31c       -6.304150E-2     $ Cu-65  AB(%) 31.521
c
C M74, submaterial 1
      13027.31c       -1.201453E-5     $ Al-27  AB(%) 100.0
       6012.31c       -2.349665E-5     $ C-12   AB(%) 98.842
       6013.31c       -2.753824E-7     $ C-13   AB(%) 1.1584
      20040.31c       -7.742686E-6     $ Ca-40  AB(%) 96.662
      20042.31c       -5.425701E-8     $ Ca-42  AB(%) 0.67736
      20043.31c       -1.159086E-8     $ Ca-43  AB(%) 0.1447
      20044.31c       -1.832560E-7

#### Generate a material from a list of zaids or elements

it is also possible to generate a material from a list of zaids. If the "natural zaids" notation is used, the elements are expanded automatically as shown in the following example.

In [13]:
from f4enix.input.materials import Material
from f4enix.input.libmanager import LibManager

libman = LibManager()  # always needed when libraries are involved
zaids = [(1000, -4.7), (5000, -30.4), (6000, -28.3), (11000, -3.2),
         (16000, -33.1), (14000, -0.06), (26000, -0.08), (7000, -0.4)]
newmat = Material.from_zaids(zaids, libman, '31c', 'header of the material')
print(newmat.to_text())

C header of the material
M1
       1001.31c       -4.698638E+0     $ H-1    AB(%) 99.971
       1002.31c       -1.361756E-3     $ H-2    AB(%) 0.028974
       5010.31c       -5.531343E+0     $ B-10   AB(%) 18.195
       5011.31c       -2.486866E+1     $ B-11   AB(%) 81.805
       6012.31c       -2.797523E+1     $ C-12   AB(%) 98.852
       6013.31c       -3.247744E-1     $ C-13   AB(%) 1.1476
      11023.31c       -3.200000E+0     $ Na-23  AB(%) 100.0
      16032.31c       -3.130391E+1     $ S-32   AB(%) 94.574
      16033.31c       -2.596888E-1     $ S-33   AB(%) 0.78456
      16034.31c       -1.530534E+0     $ S-34   AB(%) 4.624
      16036.31c       -5.866145E-3     $ S-36   AB(%) 0.017722
      14028.31c       -5.513970E-2     $ Si-28  AB(%) 91.899
      14029.31c       -2.892181E-3     $ Si-29  AB(%) 4.8203
      14030.31c       -1.968119E-3     $ Si-30  AB(%) 3.2802
      26054.31c       -4.516447E-3     $ Fe-54  AB(%) 5.6456
      26056.31c       -7.352122E-2     $ Fe-56  AB(%) 

The elements/zaids to be added can be specified with the zaid number but also using their formula as shown below

In [14]:
zaids = [('1001', -100), ('B-0', -200), ('C-12', -50)]
newmat = Material.from_zaids(zaids, libman, '31c', 'header of the material')
print(newmat.to_text())

C header of the material
M1
       1001.31c       -1.000000E+2     $ H-1    AB(%) 100.0
       5010.31c       -3.639041E+1     $ B-10   AB(%) 18.195
       5011.31c       -1.636096E+2     $ B-11   AB(%) 81.805
       6012.31c       -5.000000E+1     $ C-12   AB(%) 100.0


### Miscellanous

Another capability implemented in the MCNP inputs is the possibility to **extract a minimum working MCNP input given a set of cells**. All transformation cards, surfaces, universes and material cards related to these cells will also be extracted. Cell definitions using the # operator are also supported.

In [15]:
cells_to_extract = [34, 35, 36]
outfile = os.path.join(tempfile.gettempdir(), 'extracted.i')
inp.extract_cells(cells_to_extract, outfile)

with open(outfile, 'r') as infile:
    for line in infile:
        print(line.strip('\n'))

ITER 1D
34   7   1.0028e-001  -55 56  10 -11     imp:n,p=64
35   11  8.5299e-002  -55 56  11 -12     imp:n,p=32
36   7   1.0028e-001  -55 56  12 -13     imp:n,p=32

10   cz 447.9 
11   cz 451   
12   cz 455.2 
13   cz 458.2 
*55   pz  1000.0
*56   pz -1000.0

c ------------------------------------------------------------------
c           o-------------------------------------------------------o
C           |     S.S. 316L(N)-IG.....100 %                         |   
c           |                                                       | 
c           |     t.a.d.  = 8.5299e-002                       M11   |
C           o-------------------------------------------------------o
c
c
M11
       5010.31c        8.707160E-7     $ B-10   AB(%) 19.9
       5011.31c        3.504740E-6     $ B-11   AB(%) 80.1
c         6012.21c 7.08895E-05  $ C    (nat.)
       6012.31c        7.013100E-5     $ C-12   AB(%) 98.93
       6013.31c        7.585180E-7     $ C-13   AB(%) 1.07
c         7014.21c 2.36402

The extract_universe method allows the creation of a new MCNP input with the selected universe. The universe keywords will be erased from the cell definitions. 

In [16]:
inp_universes = Input.from_input("test_universe.i")

outfile = os.path.join(tempfile.gettempdir(), 'extracted_universe.i')
inp_universes.extract_universe(125, outfile)

Or when producing a report, it can be useful to have a summary of the tallies defined in the input

In [17]:
# Normal tallies
inp.get_tally_summary()

,Particle,Description,Normalization,Other multipliers
Tally,,,,
4,N,Neutrons Flux (energy binned) [#/cm^2],1.222e21,<NA>
6,NP,Total Nuclear Heating [W/g],1.9577e8,<NA>
14,P,Photons Flux (energy binned) [#/cm^2],1.222e21,<NA>
16,N,Neutron Heating [W/g],1.9577e8,<NA>
24,N,Fe dpa/FPY,3.8566e8,"[16, 444]"
26,P,Gamma Heating [W/g],1.9577e8,<NA>
34,N,He in 316SS appm/FPY,3.8566e10,"[11, (207:206)]"
44,N,H in 316SS appm/FPY,3.8566e10,"[11, (203:204:205)]"
54,N,T in 316SS appm/FPY,3.8566e10,"[11, 205]"


In [18]:
# FMESHES
inp.get_tally_summary(fmesh=True)

,Particle,Description,Normalization,Other multipliers
Tally,,,,
1004,N,Neutron Flux [#/cc/s],1.222e21,<NA>
1024,P,Photon Flux [#/cc/s],1.222e21,<NA>


In [19]:
inp_universes.get_cells_summary()

,material,density,universe,filler
cell,,,,
1,0,0.0,NaN,125.0
21,4,-1.0,125.0,NaN
22,0,0.0,125.0,NaN
99,0,0.0,NaN,NaN
299,0,0.0,125.0,NaN
